# Deploy Mixtral with LMI
## Hugging Face Accelerate

### 8bit quantization
See serving.properties

In [14]:
model_name = "mixtral-smoothquant"
model_filename = "trt-hf-" + model_name + ".tar.gz"
s3_prefix = "trt-hf-" + model_name
instance_type = "ml.g5.12xlarge" #"ml.g4dn.12xlarge"

In [15]:
!pip install -U sagemaker

  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.0 MB/s eta 0:00:00:00:0100:01
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.199.0
    Uninstalling sagemaker-2.199.0:
      Successfully uninstalled sagemaker-2.199.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.


In [16]:
import time
import sagemaker
from sagemaker.model import Model
from sagemaker import image_uris
from sagemaker import serializers, deserializers

role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket()
region = sess._region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [17]:
# upload model code archive to S3
!rm {model_filename} 2> /dev/null
!tar -czf {model_filename} {s3_prefix}
s3_artifact = sess.upload_data(model_filename, bucket, s3_prefix)
s3_artifact

's3://sagemaker-us-west-2-261416220256/trt-hf-mixtral-smoothquant/trt-hf-mixtral-smoothquant.tar.gz'

In [26]:
def create_model(_model_name, model_s3_url):
    # Get the LMI image uri
    image_uri = image_uris.retrieve(
        framework="djl-tensorrtllm",
        region=region,
        version="0.25.0",
        container_version="nightly"
    )
    model = Model(
        image_uri=image_uri,
        model_data=model_s3_url,
        role=role,
        name=_model_name,
        sagemaker_session=sess,
    )
    return model

In [19]:
def deploy_model(model, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name,
        container_startup_health_check_timeout=900
        #endpoint_logging=True
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [27]:
endpoint_name = model_name + "-" + time.strftime("%Y%m%d-%H%M%S")
endpoint_name

'mixtral-smoothquant-20231220-165233'

In [28]:
model = create_model(endpoint_name, s3_artifact)

In [30]:
print(model.image_uri)

763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.25.0-tensorrtllm0.5.0-cu122-nightly


In [22]:
predictor = deploy_model(model, endpoint_name)

---------------------------------------*

UnexpectedStatusException: Error hosting endpoint mixtral-smoothquant-20231220-082156: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [11]:
def test_model(prompt):
    print(predictor.predict(
        data={ 
            "inputs" : prompt,
            "parameters": { "max_length": 100 }
        }
    ))

In [12]:
test_model("What is SageMaker LMI?")

[{'generated_text': 'What is SageMaker LMI?\n\nSageMaker LMI is a tool that allows you to quickly and easily create, train, and deploy machine learning models on Amazon SageMaker. It provides a simple, intuitive interface for data scientists and developers to build, test, and deploy models without having to worry about the underlying infrastructure.\n\nSageMaker LMI supports a wide range of machine learning algorithms, including deep learning, computer vision, natural language'}]


In [ ]:
test_model("What are the recommended steps to train for an AWS Solutions Architect certification?")

In [ ]:
test_model("Name the stars in our solar system in order from fartherst to closest to the sun")

### Cleanup Resources

In [13]:
predictor.delete_endpoint()

In [14]:
model.delete_model()